# Person Re-identification using PySDK
This notebook demonstrates Person Re-Identification (Re-ID) using PySDK. Re-ID focuses on recognizing and matching people across different camera views based on their unique appearance, like clothing and body shape.

The basic pipeline works like this:
1. Detect people in the image using a person detection model.
2. Crop each detected person using the bounding box coordinates.
3. Apply the Person Re-ID model to the cropped images to extract the embeddings which can further be used to identify and match individuals across different images or camera views.

In [ ]:
import degirum as dg, degirum_tools

inference_host_address = "@local"
zoo_url = "degirum/hailo"
token = '' 
device_type = "HAILORT/HAILO8L"

# Person detection model name 
person_det_model_name = "yolov8n_relu6_person--640x640_quant_hailort_hailo8l_1"

# load AI model
person_det_model = dg.load_model(
    model_name=person_det_model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type
    
)

# Choose the Person reid model name 
person_reid_model_name = "osnet_x1_0_person_reid--256x128_quant_hailort_hailo8l_1"
# person_reid_model_name = "repvgg_a0_person_reid--256x128_quant_hailort_hailo8l_1" 

# load AI model
person_reid_model = dg.load_model(
    model_name=person_reid_model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type
    
)

#### Let's walk through a practical example of person re-identification.<br> 

Let’s walk through a hands-on example of how person re-identification (ReID) works.

We have two set of images:

 * Set A: Multiple images of the same individual, captured from different camera angles.

 * Set B: Multiple images of a different individual, also taken from various angles.

The goal is to verify whether the ReID model can:

1. Correctly match all images in Set A as the same person.

2. Correctly distinguish Set B as a different person from Set A.

This simulates a real-world scenario where the model must recognize individuals across different camera views and lighting conditions.

In [ ]:
#Image sources
image_source_1 = "../assets/person_1_multi_view.png"
image_source_2 = "../assets/person_2_multi_view.png"

# Detections
detections_1 = person_det_model(image_source_1)
detections_2 = person_det_model(image_source_2)

In [ ]:
detections_1, detections_2

In [ ]:
# Utility function for displaying crops

import matplotlib.pyplot as plt

def display_images(images, titles="Images", figsize=(15, 5)):
    """
    Display a list of images in a single row using Matplotlib.
    
    Parameters:
    - images (list): List of images (NumPy arrays) to display.
    - titles (str or list): Either a single string for overall title, or list of titles for each image.
    - figsize (tuple): Size of the figure.
    """
    num_images = len(images)
    fig, axes = plt.subplots(1, num_images, figsize=figsize)
    if num_images == 1:
        axes = [axes]  # Make iterable for single image

    for i, (ax, image) in enumerate(zip(axes, images)):
        image_rgb = image[:, :, ::-1]  # Convert BGR to RGB
        ax.imshow(image_rgb)
        ax.axis('off')
        if isinstance(titles, list) and i < len(titles):
            ax.set_title(titles[i], fontsize=12)

    if isinstance(titles, str):
        fig.suptitle(titles, fontsize=16)

    plt.tight_layout()
    plt.show()

In [ ]:
# Cropping

# Crops from the image_source_1
x1, y1, x2, y2 = map(int, detections_1.results[0]["bbox"])  # Convert bbox coordinates to integers
crop_11 = detections_1.image[y1:y2, x1:x2]  # Crop the person from the image

x1, y1, x2, y2 = map(int, detections_1.results[1]["bbox"])  # Convert bbox coordinates to integers
crop_12 = detections_1.image[y1:y2, x1:x2]  # Crop the person from the image

# Crops from the image_source_2
x1, y1, x2, y2 = map(int, detections_2.results[0]["bbox"])  # Convert bbox coordinates to integers
crop_21 = detections_2.image[y1:y2, x1:x2]  # Crop the person from the image

x1, y1, x2, y2 = map(int, detections_2.results[1]["bbox"])  # Convert bbox coordinates to integers
crop_22 = detections_2.image[y1:y2, x1:x2]  # Crop the person from the image

# Display person crops
display_images([crop_11, crop_12, crop_21, crop_22], titles=["Crop11","Crop12","Crop21", "Crop22"], figsize=(10, 5))

### Extracting embedding using a Person recognition model for each person crop

In [ ]:
#Extract the embeddings for the image_source_1 that has two crops
embedding_11 = person_reid_model(crop_11).results[0]["data"][0][0] # shape (1,512)
embedding_12 = person_reid_model(crop_12).results[0]["data"][0][0] # shape (1,512)

#Extract the embeddings for the image_source_2 that detected two crops
embedding_21 = person_reid_model(crop_21).results[0]["data"][0][0] # shape (1,512)
embedding_22 = person_reid_model(crop_22).results[0]["data"][0][0] # shape (1,512)

### Calculating cosine similarity between the embeddings

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity between crops of the same person on different camera views
similarity_1 = cosine_similarity(embedding_11, embedding_12)
print("Cosine similarity between crop11 and crop12 (two images of the same person with different camera views):",similarity_1[0][0])

similarity_1 = cosine_similarity(embedding_21, embedding_22)
print("Cosine similarity between crop21 and crop22 (two images of the same person with different camera views):", similarity_1[0][0], "\n")


# Compute cosine similarity between crops of two different people
similarity_2 = cosine_similarity(embedding_11, embedding_21)
print("Cosine similarity between crop11 and crop21 (two different people):", similarity_2[0][0])

similarity_2 = cosine_similarity(embedding_12, embedding_22)
print("Cosine similarity between crop12 and crop22 (two different people):", similarity_2[0][0])

### The results show that the cosine similarity between images of the same person (captured from different camera angles) is significantly higher than the similarity between images of two different individuals.